In [1]:
import os
from sage.manifolds.utilities import ExpressionNice as EN

In [2]:
def formal_diff(f, x):
    if f!=0:
        tempX = SR.temp_var()
        return f.subs({x: tempX}).diff(tempX).subs({tempX: x})
    if f==0:
        return 0

In [3]:
#Definimos variables y funciones
var('eps', latex_name = '\\varepsilon')
var('t')
var('r')
var('th', latex_name = '\\theta')
var('ph', latex_name = '\\varphi')
coords = (t,r,th,ph)

h = function('h')(r,th)
k = function('k')(r,th)
v = function('v')(r,th)
omega = function('omega', latex_name='\\omega')(r)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

#Definimos las funciones N, V, K y Nphi (ecuación 23 AC)
#Ver si hay que introducir eps^2/2 porque así creo que no me coinciden los uevec
Ne = e^(nu/2)*(1+1/2*eps^2*h)
Ve = e^(lamb)*(1+eps^2*v)
Ke = r^2*(1+eps^2*k)
Nphie = eps*omega

# <center> Tensor de Einstein </center>

In [4]:
gdd = matrix(SR, 4, 4)

gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

guu = 1/gdd

K1 = matrix(SR, 4, 4)
K1[0,3] = -omega*r^2*sin(th)^2
K1[3,0] = -omega*r^2*sin(th)^2

K2 = matrix(SR, 4, 4)
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*v
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

In [5]:
gedd = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        gedd[ii,jj] = gdd[ii,jj] + eps*K1[ii,jj] + 1/2*eps^2*K2[ii,jj]
        
geuu = 1/gedd

chris = [[[0 for kk in range(4)] for jj in range(4)] for ii in range(4)]
for ii in range(4):
    for jj in range(4):
        for kk in range(4):
            chris[ii][jj][kk] = 0.5*sum(geuu[ii,dd]*(formal_diff(gedd[dd,jj],coords[kk])
                                                   + formal_diff(gedd[dd,kk],coords[jj])
                                                   - formal_diff(gedd[jj,kk],coords[dd])) for dd in range(4))

riem = [[[[0 for ll in range(4)] for kk in range(4)] for jj in range(4)] for ii in range(4)]
for ii in range(4):
    for jj in range(4):
        for kk in range(4):
            for ll in range(4):
                riem[ii][jj][kk][ll] = (formal_diff(chris[ii][jj][ll],coords[kk])
                                      - formal_diff(chris[ii][jj][kk],coords[ll])
                                      + sum(chris[ii][kk][dd]*chris[dd][ll][jj]
                                          - chris[ii][ll][dd]*chris[dd][kk][jj] for dd in range(4)))

ricci = matrix(SR, 4, 4)
for ii in range(4):
    for kk in range(4):
        ricci[ii,kk] = sum(riem[dd][ii][dd][kk] for dd in range(4))

s_curv = sum(sum(ricci[ii,jj]*geuu[ii,jj] for ii in range(4)) for jj in range(4))

Ge = matrix(SR, 4, 4)
Ge = ricci - 0.5*gedd*s_curv

dGe = matrix(SR, 4, 4)
ddGe = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        dGe[ii,jj] = formal_diff(Ge[ii,jj],eps)
        ddGe[ii,jj] = formal_diff(formal_diff(Ge[ii,jj],eps),eps)

# <center> Tensor de energía-momento </center>

In [6]:
#Funciones dependientes de eps
Omegane = function('Omegane', latex_name='\\Omega_{n\\varepsilon}')(eps)
Omegape = function('Omegape', latex_name='\\Omega_{p\\varepsilon}')(eps)

Nne = function('Nne', latex_name='N_{n\\varepsilon}')(r,th,eps)
Npe = function('Npe', latex_name='N_{p\\varepsilon}')(r,th,eps)

#Tomando eps=0
Nn = function('Nn', latex_name='N_{n}')(r,th)
Np = function('Np', latex_name='N_{p}')(r,th)

sus_func = [eps==0,
            Omegane.subs(eps==0)==0, #Staticity at the background
            Nne.subs(eps==0)==Nn,
            Omegape.subs(eps==0)==0, #Staticity at the background
            Npe.subs(eps==0)==Np]

## <center> Campo $u$ de los neutrones </center>

In [7]:
Nne_paper = 1/sqrt(Ne^2-sin(th)^2*Ke*(Nphie-Omegane)^2)

uevec_paper = [Nne_paper,0,0,Nne_paper*Omegane]

In [8]:
uevec = [Nne,0,0,Nne*Omegane]
Uevec = [0,0,0,0]

for ii in range(4):
    for jj in range(4):
        Uevec[ii] += uevec[jj]*gedd[ii,jj]

norm = 0
for ii in range(4):
    norm += uevec[ii]*Uevec[ii]

Nne_value = sqrt(solve(norm==-1, Nne^2)[0].rhs())

for ii in range(4):
    uevec[ii] = uevec[ii].subs(Nne==Nne_value)
    Uevec[ii] = Uevec[ii].subs(Nne==Nne_value)

Nn_value = copy(Nne_value)
for ii in range(len(sus_func)):
    Nn_value = Nn_value.subs(sus_func[ii])

sus_func.extend([Nn==Nn_value])

uvec = copy(uevec)
for ii in range(4):
    for jj in range(len(sus_func)):
        uvec[ii] = uvec[ii].subs(sus_func[jj])

Nne_paper = 1/sqrt(Ne^2 - sin(th)^2*Ke*(Nphie-Omegane)^2)

uvec_paper = [Nne_paper,0,0,Omegane*Nne_paper]
for ii in range(4):
    for jj in range(len(sus_func)):
        uvec_paper[ii] = uvec_paper[ii].subs(sus_func[jj])

uvec_check = [x-y for (x,y) in zip(uvec, uvec_paper)]

show(LatexExpr("u_{\\varepsilon}\\lvert_{\\varepsilon=0} \\rightarrow "), bool(uvec_check==[0,0,0,0]))

u_{\varepsilon}\lvert_{\varepsilon=0} \rightarrow  True

In [9]:
duevec, duevec_paper, duevec_check = [0,0,0,0], [0,0,0,0], [0,0,0,0]

for ii in range(4):
    duevec[ii] = formal_diff(uevec[ii], eps)
    duevec_paper[ii] = formal_diff(uevec_paper[ii], eps)
    duevec_check[ii] = duevec[ii] - duevec_paper[ii]
    for jj in range(len(sus_func)):
        duevec_check[ii] = duevec_check[ii].subs(sus_func[jj])

show(LatexExpr("\\frac{d u_{\\varepsilon}}{d \\varepsilon}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(duevec_check==[0,0,0,0]))

\frac{d u_{\varepsilon}}{d \varepsilon}\bigg\lvert_{\varepsilon=0} \rightarrow  True

In [10]:
duevec, duevec_paper, duevec_check = [0,0,0,0], [0,0,0,0], [0,0,0,0]

for ii in range(4):
    duevec[ii] = formal_diff(uevec[ii], eps)
    duevec_paper[ii] = formal_diff(uevec_paper[ii], eps)
    duevec_check[ii] = duevec[ii] - duevec_paper[ii]
    for jj in range(len(sus_func)):
        duevec_check[ii] = duevec_check[ii].subs(sus_func[jj])

show(LatexExpr("\\frac{d u_{\\varepsilon}}{d \\varepsilon}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(duevec_check==[0,0,0,0]))

\frac{d u_{\varepsilon}}{d \varepsilon}\bigg\lvert_{\varepsilon=0} \rightarrow  True

In [11]:
dduevec, dduevec_paper, dduevec_check = [0,0,0,0], [0,0,0,0], [0,0,0,0]

for ii in range(4):
    dduevec[ii] = formal_diff(duevec[ii], eps)
    dduevec_paper[ii] = formal_diff(duevec_paper[ii], eps)
    dduevec_check[ii] = dduevec[ii] - dduevec_paper[ii]
    for jj in range(len(sus_func)):
        dduevec_check[ii] = dduevec_check[ii].subs(sus_func[jj])

show(LatexExpr("\\frac{d^2 u_{\\varepsilon}}{d \\varepsilon^2}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(dduevec_check==[0,0,0,0]))
show(dduevec_check[0].simplify_full())

\frac{d^2 u_{\varepsilon}}{d \varepsilon^2}\bigg\lvert_{\varepsilon=0} \rightarrow  False

-e^(-1/2*nu(r))*h(r, th)

In [12]:
var('Omegan1', latex_name="\\Omega^{(1)}_{n}")

sus_func.extend([formal_diff(formal_diff(Omegane,eps),eps).subs(eps==0) == 0, #En nuestro gauge Omega2=0
                 formal_diff(Omegane,eps).subs(eps==0) == Omegan1])

uvec1 = [0,0,0,0]
uvec2 = [0,0,0,0]

for ii in range(4):
    uvec1[ii] = formal_diff(uevec[ii],eps)
    uvec2[ii] = formal_diff(formal_diff(uevec[ii],eps),eps)

for ii in range(4):
    for jj in range(len(sus_func)):
        uvec1[ii] = uvec1[ii].subs(sus_func[jj])
        uvec2[ii] = uvec2[ii].subs(sus_func[jj])

uvec1_paper = [0,0,0,0]
uvec1_paper[0] = 1/2*e^(-3*nu/2)*K1[0,0]
uvec1_paper[3] = e^(-nu/2)*Omegan1

uvec2_paper = [0,0,0,0]
uvec2_paper[0] = e^(-3*nu/2)*(1/2*K2[0,0]+3/4*e^(-nu)*K1[0,0]+2*Omegan1*K1[0,3]+Omegan1^2*r^2*sin(th)^2)

uvec1_check = [ii-jj for (ii,jj) in zip(uvec1, uvec1_paper)]
uvec2_check = [ii-jj for (ii,jj) in zip(uvec2, uvec2_paper)]

show(LatexExpr("u^{(1)} \\rightarrow"), uvec1_check==[0,0,0,0])
show(LatexExpr("u^{(2)} \\rightarrow"), uvec2_check==[0,0,0,0])

u^{(1)} \rightarrow True

u^{(2)} \rightarrow True

## <center> Campo $v$ de los protones </center>

In [13]:
Npe_paper = 1/sqrt(Ne^2-sin(th)^2*Ke*(Nphie-Omegape)^2)

vevec_paper = [Npe_paper,0,0,Npe_paper*Omegape]

In [14]:
vevec = [Npe,0,0,Npe*Omegape]
Vevec = [0,0,0,0]

for ii in range(4):
    for jj in range(4):
        Vevec[ii] += vevec[jj]*gedd[ii,jj]

norm = 0
for ii in range(4):
    norm += vevec[ii]*Vevec[ii]

Npe_value = sqrt(solve(norm==-1, Npe^2)[0].rhs())
for ii in range(4):
    vevec[ii] = vevec[ii].subs(Npe==Npe_value)
    Vevec[ii] = Vevec[ii].subs(Npe==Npe_value)

Np_value = copy(Npe_value)
for ii in range(len(sus_func)):
    Np_value = Np_value.subs(sus_func[ii])

sus_func.extend([Np==Np_value])

vvec = copy(vevec)
for ii in range(4):
    for jj in range(len(sus_func)):
        vvec[ii] = vvec[ii].subs(sus_func[jj])

Npe_paper = 1/sqrt(Ne^2 - sin(th)^2*Ke*(Nphie-Omegape)^2)

vvec_paper = [Npe_paper,0,0,Omegape*Npe_paper]

for ii in range(4):
    for jj in range(len(sus_func)):
        vvec_paper[ii] = vvec_paper[ii].subs(sus_func[jj])

vvec_check = [x-y for (x,y) in zip(vvec, vvec_paper)]

show(LatexExpr("v_{\\varepsilon} \\rightarrow "), bool(vvec_check==[0,0,0,0]))

v_{\varepsilon} \rightarrow  True

In [15]:
dvevec, dvevec_paper, dvevec_check = [0,0,0,0], [0,0,0,0], [0,0,0,0]

for ii in range(4):
    dvevec[ii] = formal_diff(vevec[ii], eps)
    dvevec_paper[ii] = formal_diff(vevec_paper[ii], eps)
    dvevec_check[ii] = dvevec[ii] - dvevec_paper[ii]
    for jj in range(len(sus_func)):
        dvevec_check[ii] = dvevec_check[ii].subs(sus_func[jj])

show(LatexExpr("\\frac{d v_{\\varepsilon}}{d \\varepsilon}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(dvevec_check==[0,0,0,0]))

\frac{d v_{\varepsilon}}{d \varepsilon}\bigg\lvert_{\varepsilon=0} \rightarrow  True

In [16]:
ddvevec, ddvevec_paper, ddvevec_check = [0,0,0,0], [0,0,0,0], [0,0,0,0]

for ii in range(4):
    ddvevec[ii] = formal_diff(dvevec[ii], eps)
    ddvevec_paper[ii] = formal_diff(dvevec_paper[ii], eps)
    ddvevec_check[ii] = ddvevec[ii] - ddvevec_paper[ii]
    for jj in range(len(sus_func)):
        ddvevec_check[ii] = ddvevec_check[ii].subs(sus_func[jj])

show(LatexExpr("\\frac{d^2 v_{\\varepsilon}}{d \\varepsilon^2}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(ddvevec_check==[0,0,0,0]))
show(dduevec_check[0].simplify_full())

\frac{d^2 v_{\varepsilon}}{d \varepsilon^2}\bigg\lvert_{\varepsilon=0} \rightarrow  False

-e^(-1/2*nu(r))*h(r, th)

In [17]:
var('Omegap1', latex_name="\\Omega^{(1)}_{p}")

sus_func.extend([formal_diff(formal_diff(Omegape,eps),eps).subs(eps==0) == 0, #En nuestro gauge Omega2=0
                 formal_diff(Omegape,eps).subs(eps==0) == Omegap1])

vvec1 = [0,0,0,0]
vvec2 = [0,0,0,0]

for ii in range(4):
    vvec1[ii] = formal_diff(vevec[ii],eps)
    vvec2[ii] = formal_diff(formal_diff(vevec[ii],eps),eps)

for ii in range(4):
    for jj in range(len(sus_func)):
        vvec1[ii] = vvec1[ii].subs(sus_func[jj])
        vvec2[ii] = vvec2[ii].subs(sus_func[jj])

vvec1_paper = [0,0,0,0]
vvec1_paper[0] = 1/2*e^(-3*nu/2)*K1[0,0]
vvec1_paper[3] = e^(-nu/2)*Omegap1

vvec2_paper = [0,0,0,0]
vvec2_paper[0] = e^(-3*nu/2)*(1/2*K2[0,0]+3/4*e^(-nu)*K1[0,0]+2*Omegap1*K1[0,3]+Omegap1^2*r^2*sin(th)^2)
vvec2_paper[3] = e^(-nu/2)*(e^(-nu)*K1[0,0]*Omegap1)  

vvec1_check = [ii-jj for (ii,jj) in zip(vvec1, vvec1_paper)]
vvec2_check = [ii-jj for (ii,jj) in zip(vvec2, vvec2_paper)]

show(LatexExpr("v^{(1)} \\rightarrow"), vvec1_check==[0,0,0,0])
show(LatexExpr("v^{(2)} \\rightarrow"), vvec2_check==[0,0,0,0])

v^{(1)} \rightarrow True

v^{(2)} \rightarrow True

## <center> Ecuación de Einstein </center>

In [18]:
n = function('n')(r,th,eps)
p = function('p')(r,th,eps)

n0 = function('n0', latex_name="n_0")(r)
p0 = function('p0', latex_name="p_0")(r)

eta = function('eta', latex_name="\\eta")(r,th)
Phi = function('Phi', latex_name="\\Phi")(r,th)

n2 = function('n2')(r,th,eps)
p2 = function('p2')(r,th,eps)
x2 = function('x2')(r,th,eps)

n = n0*(1 + 1/2*eps^2*eta)
p = p0*(1 + 1/2*eps^2*Phi)

sus_func.extend([n2==n^2,p2==p^2,x2==n*p])

Lambdae = function('Lambdae', latex_name='\\Lambda_{\\varepsilon}')(n2, p2, x2)
Psie = function('Psie', latex_name='\\Psi_{\\varepsilon}')(n2, p2, x2)

nevec = [n*ii for ii in uevec] #eq 13 AC
pevec = [p*ii for ii in vevec] #eq 13 AC

ned = [n*ii for ii in Uevec]
ped = [p*ii for ii in Vevec]

nd = copy(ned)
pd = copy(ped)
for ii in range(4):
    for jj in range(len(sus_func)):
        nd[ii] = nd[ii].subs(sus_func[jj])
        pd[ii] = pd[ii].subs(sus_func[jj])

A = -formal_diff(Lambdae, x2)
B = -2*formal_diff(Lambdae, n^2)
C = -2*formal_diff(Lambdae, p^2)

mued = [B*ii for ii in ned] + [A*jj for jj in ped] #eq 7 AC
chied = [C*ii for ii in ped] + [A*jj for jj in ned] #eq 7 AC

muevec = [0,0,0,0]
chievec = [0,0,0,0]
for ii in range(4):
    for jj in range(4):
        muevec[ii] += mued[jj]*geuu[ii,jj]
        chievec[ii] += chied[jj]*geuu[ii,jj]

Psie = Lambdae - sum([ii*jj for (ii,jj) in zip(nevec, mued)]) - sum([ii*jj for (ii,jj) in zip(pevec, chied)])

LambdaePsie = Psie - Lambdae

In [19]:
dirac = diagonal_matrix([1,1,1,1])

Teud = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        Teud[ii,jj] = Psie*dirac[ii,jj] + pevec[ii]*chied[jj] + nevec[ii]*mued[jj]

Te, dTe, ddTe = matrix(SR, 4, 4), matrix(SR, 4, 4), matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        for kk in range(4):
            Te[ii,jj] += Teud[kk,jj]*gedd[kk,ii]

        dTe[ii,jj] = formal_diff(Te[ii,jj],eps)
        ddTe[ii,jj] = formal_diff(formal_diff(Te[ii,jj],eps),eps)

kappa = 8*pi

In [20]:
mue = B*n + A*p #eq 22 C
chie = A*n + C*p #eq 22 C

#Volvemos a definir mu_d y chi_d porque los que habíamos definido antes eran de AC y este es de C
mued_C = [mue*ii for ii in Uevec] #eq 17 C
chied_C = [chie*ii for ii in Uevec] #eq 17 C

muinf = -mued_C[0] #eq 20
chiinf = -chied_C[0] #eq 20
for ii in range(len(sus_func)):
    muinf = muinf.subs(sus_func[ii])
    chiinf = chiinf.subs(sus_func[ii])

#eq_mu = formal_diff(muinf,r)
#eq_chi = formal_diff(chiinf,r)
#for ii in range(len(sus_func)):
#    eq_mu = eq_mu.subs(sus_func[ii])
#    eq_chi = eq_chi.subs(sus_func[ii])

A0, B0, C0 = copy(A), copy(B), copy(C)

A00 = A + 2*formal_diff(B,p^2)*n*p + 2*formal_diff(A,n^2)*n^2 + 2*formal_diff(A,p^2)*p^2 + formal_diff(A,x^2)*p*n
B00 = B + 2*formal_diff(B,n^2)*n^2 + 4*formal_diff(A,n^2)*n*p + formal_diff(A,x^2)*p^2
C00 = C + 2*formal_diff(C,p^2)*p^2 + 4*formal_diff(A,p^2)*n*p + formal_diff(A,x^2)*n^2

A000, B000, C000 = A00, B00, C00
for ii in range(len(sus_func)):
    A0 = A0.subs(sus_func[ii])
    B0 = B0.subs(sus_func[ii])
    C0 = C0.subs(sus_func[ii])
    A000 = A000.subs(sus_func[ii])
    B000 = B000.subs(sus_func[ii])
    C000 = C000.subs(sus_func[ii])

#eq26_1 = A000*formal_diff(p0,r) + B000*formal_diff(n0,r) + 1/2*(B0*n0 + A0*p0)*formal_diff(nu,r)
#eq26_2 = C000*formal_diff(p0,r) + A000*formal_diff(n0,r) + 1/2*(A0*n0 + C0*p0)*formal_diff(nu,r)
#for ii in range(len(sus_func)):
#    eq26_1 = eq26_1.subs(sus_func[ii])
#    eq26_2 = eq26_2.subs(sus_func[ii])

#eq_mu = eq_mu*e^(-nu/2)
#eq_mu_check = eq_mu - eq26_1
#eq_chi = eq_chi*e^(-nu/2)
#eq_chi_check = eq_chi - eq26_2
#
#show(LatexExpr("\\text{Eq 26.1} \\rightarrow"), bool(eq_mu_check==0))
#show(LatexExpr("\\text{Eq 26.2} \\rightarrow"), bool(eq_chi_check==0))

In [21]:
#dp0_value = solve(eq26_1, formal_diff(p0,r))[0].rhs()
#for ii in range(len(sus_func)):
#    dp0_value = dp0_value.subs(sus_func[ii])
#sus_func.extend([formal_diff(p,r).subs(eps==0)==dp0_value])
#
#dn0_value = solve(eq26_2, diff(n0,r))[0].rhs()
#for ii in range(len(sus_func)):
#    dn0_value = dn0_value.subs(sus_func[ii])
#sus_func.extend([formal_diff(n,r).subs(eps==0)==dn0_value])

# <center> $G_{\mu\nu} = \kappa T_{\mu\nu}$ </center>

In [22]:
Lambda0 = copy(Lambdae)
Psi0 = copy(Psie)
for ii in range(len(sus_func)):
    Lambda0 = Lambda0.subs(sus_func[ii])
    Psi0 = Psi0.subs(sus_func[ii])

G = copy(Ge)
T = copy(Te)
for ii in range(len(sus_func)):
    G = G.subs(sus_func[ii])
    T = T.subs(sus_func[ii])

## <center> $G_{00} = \kappa T_{00}$ </center>

In [23]:
dlamb_value = solve(G[0,0] - kappa*T[0,0] == 0, formal_diff(lamb, r))[0].rhs().simplify_full()
dlamb_paper = (1-e^(lamb))/r - 8*pi*r*e^(lamb)*Lambda0
dlamb_check = dlamb_value - dlamb_paper
show(LatexExpr("\\lambda' \\rightarrow"), bool(dlamb_check==0))

for ii in range(len(sus_func)):
    dlamb_value = dlamb_value.subs(sus_func[ii])

sus_func.extend([formal_diff(lamb,r)==dlamb_value])

\lambda' \rightarrow True

## <center> $G_{11} = \kappa T_{11}$ </center>


In [24]:
dnu_value_11 = solve(G[1,1] - kappa*T[1,1] == 0, formal_diff(nu, r))[0].rhs().simplify_full()
dnu_paper = -(1-e^(lamb))/r + 8*pi*r*e^(lamb)*Psi0
dnu_check_11 = dnu_value_11 - dnu_paper
show(LatexExpr("\\nu' \\rightarrow"), bool(dnu_check_11==0))

for ii in range(len(sus_func)):
    dnu_value_11 = dnu_value_11.subs(sus_func[ii])

sus_func.extend([formal_diff(nu,r)==dnu_value_11])

\nu' \rightarrow True

In [25]:
#ddnu_value_11 = formal_diff(dnu_value_11,r).simplify_full()
#for ii in range(len(sus_func)):
#    ddnu_value_11 = ddnu_value_11.subs(sus_func[ii])
#
#ddnu_paper = 2*r*formal_diff(formal_diff(nu,r),r)\
#           + formal_diff(nu,r)*(r*formal_diff(nu,r)-2)\
#           - formal_diff(lamb,r)*(2+r*formal_diff(nu,r))\
#           + 4/r*(e^(lamb)-1) == 0
#
#ddnu_paper = solve(ddnu_paper, formal_diff(formal_diff(nu,r),r))[0].rhs().simplify_full()
#for ii in range(len(sus_func)):
#    ddnu_value_11 = ddnu_value_11.subs(sus_func[ii])
#    ddnu_paper = ddnu_paper.subs(sus_func[ii])
#
#ddnu_check_11 = ddnu_value_11 - ddnu_paper
#show(ddnu_value_11)
#show(ddnu_paper)
#show(LatexExpr("\\nu' \\rightarrow"), bool(ddnu_check_11==0))

## <center> $G_{11} = \kappa T_{11}$ y $G_{22} = \kappa T_{22}$ </center>


In [26]:
dnu_value_22 = solve(G[2,2] - kappa*T[2,2] == 0, formal_diff(nu, r))[0].rhs().simplify_full()

#Despejamos la raiz cuadrada de la ecuación
dEq_1122 = -(dnu_value_11 - dnu_value_22 == 0)*e^(nu)*2*r
dEq_1122 = dEq_1122 + sqrt(-256*pi*r^2*e^(lamb + 2*nu)*n0^2*formal_diff(Lambdae,n^2).subs(eps==0)\
                           -256*pi*r^2*e^(lamb + 2*nu)*p0^2*formal_diff(Lambdae,p^2).subs(eps==0)\
                           +r^2*e^(2*nu)*formal_diff(lamb, r)^2\
                           -8*r^2*e^(2*nu)*formal_diff(formal_diff(nu, r), r)\
                           +4*r*e^(2*nu)*formal_diff(lamb, r)\
                           +4*(32*pi*r^2*Lambda0*e^lamb + 1)*e^(2*nu))

for ii in range(len(sus_func)):
    dEq_1122 = dEq_1122.subs(sus_func[ii])

ddnu_value = solve(dEq_1122^2, formal_diff(formal_diff(nu,r),r))[0].rhs().simplify_full()

ddnu_paper = 2*r*formal_diff(formal_diff(nu,r),r)\
           + formal_diff(nu,r)*(r*formal_diff(nu,r)-2)\
           - formal_diff(lamb,r)*(2+r*formal_diff(nu,r))\
           + 4/r*(e^(lamb)-1) == 0
ddnu_paper = solve(ddnu_paper, formal_diff(formal_diff(nu,r),r))[0].rhs()
for ii in range(len(sus_func)):
    ddnu_paper = ddnu_paper.subs(sus_func[ii])
    
ddnu_check = ddnu_value - ddnu_paper
show(LatexExpr("\\nu'' \\rightarrow"), bool(ddnu_check==0))

sus_func.extend([formal_diff(formal_diff(nu,r),r)==ddnu_value])

\nu'' \rightarrow True

# <center> $G^{\prime}_{\mu\nu} = \kappa T^{\prime}_{\mu\nu}$ </center>

In [27]:
dLambdaePsie = formal_diff(LambdaePsie, eps)
for ii in range(len(sus_func)):
    dLambdaePsie = dLambdaePsie.subs(sus_func[ii])

In [28]:
j = e^(-(lamb+nu)/2)
M = r*(1-e^(-lamb))/2

dG = copy(dGe)
dT = copy(dTe)
for ii in range(0,len(sus_func)):
    dG = dG.subs(sus_func[ii])
    dT = dT.subs(sus_func[ii])

In [29]:
show(EN(dT[0,3].simplify_full()))

-r^2*Lambdae(n2(r, th, 0), p2(r, th, 0), x2(r, th, 0))*omega(r)*sin(th)^2

In [30]:
Lambda1_value = formal_diff(Lambdae, eps)
for ii in range(len(sus_func)):
    Lambda1_value = Lambda1_value.subs(sus_func[ii])

Lambda1_paper = 0
Lambda1_check = Lambda1_value - Lambda1_paper

show(LatexExpr("\\Lambda^{(1)} = "), Lambda1_value, LatexExpr("\\rightarrow"), bool(Lambda1_check==0))

\Lambda^{(1)} =  D[0](Lambdae)(n2(r, th, 0), p2(r, th, 0), x2(r, th, 0))*D[2](n2)(r, th, 0) + D[1](Lambdae)(n2(r, th, 0), p2(r, th, 0), x2(r, th, 0))*D[2](p2)(r, th, 0) + D[2](Lambdae)(n2(r, th, 0), p2(r, th, 0), x2(r, th, 0))*D[2](x2)(r, th, 0) \rightarrow False

In [31]:
Psi1_value = formal_diff(Lambdae, eps) + formal_diff(LambdaePsie, eps)
for ii in range(len(sus_func)):
    Psi1_value = Psi1_value.subs(sus_func[ii])

Psi1_paper = 0
Psi1_check = Psi1_value - Psi1_paper

show(LatexExpr("\\Psi^{(1)} = "), Psi1_value, LatexExpr("\\rightarrow"), bool(Psi1_check==0))

\Psi^{(1)} =  D[0](Lambdae)(n2(r, th, 0), p2(r, th, 0), x2(r, th, 0))*D[2](n2)(r, th, 0) + D[1](Lambdae)(n2(r, th, 0), p2(r, th, 0), x2(r, th, 0))*D[2](p2)(r, th, 0) + D[2](Lambdae)(n2(r, th, 0), p2(r, th, 0), x2(r, th, 0))*D[2](x2)(r, th, 0) \rightarrow False

In [32]:
#De dEq el único valor que no es idénticamente nulo es 03 (y 30)
dEq_03 = dG[0,3] - kappa*dT[0,3] == 0

ddomega_value = solve(dEq_03, formal_diff(formal_diff(omega,r),r))[0].rhs().simplify_full()

Ln = omega - Omegan1
Lp = omega - Omegap1

chi0 = C*p + A*n
for ii in range(len(sus_func)):
    chi0 = chi0.subs(sus_func[ii])

ddomega_paper = 1/(r^4)*formal_diff(r^4*e^(-(lamb+nu)/2)*formal_diff(Ln,r),r)\
              - 16*pi*e^((lamb-nu)/2)*(Psi0-Lambda0)*Ln\
              - 16*pi*e^((lamb-nu)/2)*chi0*p0*(Omegan1-Omegap1) == 0
ddomega_paper = solve(ddomega_paper, formal_diff(formal_diff(omega,r),r))[0].rhs().simplify_full()

for ii in range(len(sus_func)):
    ddomega_value = ddomega_value.subs(sus_func[ii])
    ddomega_paper = ddomega_paper.subs(sus_func[ii])

ddomega_check = ddomega_value - ddomega_paper
for ii in range(len(sus_func)):
    ddomega_check = ddomega_check.subs(sus_func[ii])

ddomega_check = ddomega_check.simplify_full()

show(LatexExpr("\\Delta\\omega'' \\rightarrow "), bool(ddomega_check==0))

show(EN(ddomega_paper.simplify_full()))
show(EN(ddomega_value.simplify_full()))
show(EN(ddomega_check))

#ddomega_value debería tener un término proporcional a Omegan1-Omegap1

\Delta\omega'' \rightarrow  False

-4*(4*(pi*Omegan1 - pi*Omegap1)*r*e^lamb(r)*n0(r)*p0(r)*d(Lambdae)/d(x2(r, th, 0)) + d(omega)/dr)/r

-4*d(omega)/dr/r

16*(pi*Omegan1 - pi*Omegap1)*e^lamb(r)*n0(r)*p0(r)*d(Lambdae)/d(x2(r, th, 0))

In [33]:
sys.exit()

SystemExit: 

In [ ]:
f = r^4*Omegane*Omegape^2*e^nu*n0^2*formal_diff(Lambdae,n0^2)

#show(EN(f))

fprima = formal_diff(f,eps).subs(eps==0)
show(EN(fprima))

In [ ]:
show(EN(formal_diff(Lambdae,eps).simplify_full()))

In [ ]:
cosados = Omegane*Lambdae
show(EN(formal_diff(cosados,eps).subs(eps==0)))

In [ ]:
show(EN((formal_diff(Lambdae,eps).simplify_full())))

In [ ]:
cosa = dTe
for ii in range(len(sus_func)):
    cosa = cosa.subs(sus_func[ii])

In [ ]:
sys.exit()

# <center> $G^{\prime\prime}_{\mu\nu} = \kappa T^{\prime\prime}_{\mu\nu}$ </center>

In [ ]:
ddLambdaePsie = diff(LambdaePsie, eps, eps)
for ii in range(len(sus_func)):
    ddLambdaePsie = ddLambdaePsie.subs(sus_func[ii])

Lambda2_value = formal_diff(formal_diff(Lambdae, eps),eps)
Psi2_value = Lambda2_value + ddLambdaePsie
for ii in range(len(sus_func)):
    Lambda2_value = Lambda2_value.subs(sus_func[ii])
    Psi2_value = Psi2_value.subs(sus_func[ii])

In [ ]:
ddG = copy(ddGe)
ddT = copy(ddTe)
for ii in range(len(sus_func)):
    ddG = ddG.subs(sus_func[ii])
    ddT = ddT.subs(sus_func[ii])

## <center> Descomposición en $P_l(\cos(\theta))$ </center>

In [ ]:
h0 = function('h0', latex_name='h_0')(r)
v0 = function('v0', latex_name='v_0')(r)
eta0 = function('eta0', latex_name='\\eta_0')(r)
Phi0 = function('Phi0', latex_name='\\Phi_0')(r)

h2 = function('h2', latex_name='h_2')(r)
k2 = function('k2', latex_name='k_2')(r)
v2 = function('v2', latex_name='v_2')(r)
eta2 = function('eta2', latex_name='\\eta_2')(r)
Phi2 = function('Phi2', latex_name='\\Phi_2')(r)

LP1 = function('LP1', latex_name='LP_1')(th)
LP2 = function('LP2', latex_name='LP_2')(th)

lp2 = (3*cos(th)^2-1)/2

In [ ]:
s_v(r,th) = v0+v2*lp2
s_k(r,th) = k2*lp2
s_h(r,th) = h0+h2*lp2
s_eta(r,th) = eta0+eta2*lp2
s_Phi(r,th) = Phi0+Phi2*lp2

sus_descomp = [function('v'), s_v,
               function('k'), s_k,
               function('h'), s_h,
               function('eta', latex_name="\\eta"), s_eta,
               function('Phi', latex_name="\\Phi"), s_Phi]

for ii in range(4):
    for jj in range(4):
        for kk in range(0, len(sus_descomp), 2):
            ddG[ii,jj] = ddG[ii,jj].substitute_function(sus_descomp[kk], sus_descomp[kk+1])
            ddT[ii,jj] = ddT[ii,jj].substitute_function(sus_descomp[kk], sus_descomp[kk+1])

ddG = ddG.simplify_full()
ddT = ddT.simplify_full()

In [ ]:
sus_legendre = [sin(th)^2 == (2/3)*(1-LP2),
                1/sin(th)^2 == 1/((2/3)*(1-LP2)),
                cos(th)^2 == (2*LP2+1)/3,
                1/cos(th)^2 == 1/(2*LP2+1)/3,
                cos(th) == LP1]

### <center> $\boxed{l=0}$ </center>

Ecuación (38)

In [ ]:
Lambda2 = formal_diff(formal_diff(Lambdae, eps),eps).simplify_full()

for ii in range(len(sus_func)):
    Lambda2 = Lambda2.subs(sus_func[ii])
for ii in range(0, len(sus_descomp), 2):
    Lambda2 = Lambda2.substitute_function(sus_descomp[ii], sus_descomp[ii+1])
for ii in range(len(sus_legendre)):
    Lambda2 = Lambda2.subs(sus_legendre[ii])
Lambda20 = Lambda2.subs(LP2==0)

mu0 = B0*n0 + A0*p0
Omegan2, Omegap2 = 0, 0
Lambda20_paper = mu0*n0*eta0 + chi0*p0*Phi0 + r^2/(3*e^(nu))*A0*n0*p0*(Omegan1-Omegap1)^2
Lambda20_paper = -Lambda20_paper
for ii in range(len(sus_func)):
    Lambda20_paper = Lambda20_paper.subs(sus_func[ii])

Lambda20_check = (Lambda20 - Lambda20_paper).simplify_full()

show(LatexExpr("\\Lambda^{(2)}_0 \\rightarrow"), bool(Lambda20_check==0))

show(EN(Lambda20_check))

Ecuación (42.1)

In [ ]:
var('gamman', latex_name = "\\gamma_n")

gamman = formal_diff(formal_diff(mue,eps),eps)/muinf
gamman_paper = + B000*n0*eta0/mu0\
               + A000*p0*Phi0/mu0\
               + r^2/(3*e^(nu))*p0/mu0*(A0+n0*formal_diff(A,n) + n0*p0*formal_diff(A,x2))*(Omegan1-Omegap1)^2\
               - r^2/(3*e^nu)*Ln^2 + h0
gamman_check = gamman - gamman_paper
for ii in range(len(sus_func)):
    gamman = gamman.subs(sus_func[ii])
    gamman_paper = gamman_paper.subs(sus_func[ii])
    gamman_check = gamman_check.subs(sus_func[ii])

gamman = gamman.simplify_full()

for ii in range(0,len(sus_descomp),2):
    gamman = gamman.substitute_function(sus_descomp[ii],sus_descomp[ii+1])
for ii in range(len(sus_legendre)):
    gamman = gamman.subs(sus_legendre[ii])

gamman = gamman.simplify_full()
gamman = gamman.subs(LP2==0).simplify_full()

gamman_paper = gamman_paper.simplify_full()
gamman_check = gamman_check.simplify_full()

show(LatexExpr("\\gamma_n \\rightarrow"), bool(gamman_check==0))

show(EN(gamman_check))

In [ ]:
sys.exit()

Ecuación (42.2)

In [ ]:
var('gammap', latex_name = "\\gamma_p")

gammap = formal_diff(formal_diff(mue,eps),eps)/muinf
gammap_paper = + C000*p0*Phi0/chi0\
               + A000*n0*eta0/chi0\
               + r^2/(3*e^(nu))*n0/chi0*(A0+p0*formal_diff(A,p) + n0*p0*formal_diff(A,x2))*(Omegan1-Omegap1)^2\
               - r^2/(3*e^nu)*Lp^2 + h0
gammap_check = gammap - gammap_paper
for ii in range(len(sus_func)):
    gammap = gammap.subs(sus_func[ii])
    gammap_paper = gammap_paper.subs(sus_func[ii])
    gammap_check = gammap_check.subs(sus_func[ii])

gammap = gammap.simplify_full()

for ii in range(0,len(sus_descomp),2):
    gammap = gammap.substitute_function(sus_descomp[ii],sus_descomp[ii+1])
for ii in range(len(sus_legendre)):
    gammap = gammap.subs(sus_legendre[ii])

gammap = gammap.simplify_full()
gammap = gammap.subs(LP2==0).simplify_full()

gammap_paper = gammap_paper.simplify_full()
gammap_check = gammap_check.simplify_full()

show(LatexExpr("\\gamma_p \\rightarrow"), bool(gammap_check==0))

show(EN(gammap_check))

Ecuación (47)

In [ ]:
ddEq_00 = ddG[0,0] - kappa*ddT[0,0]

ddEq_00_paper = 16*pi*r^2/(3*e^(nu))*((Psi0-Lambda0)*Ln^2\
                                      + chi0*p0*(Omegan1-Omegap1)*(Ln+Lp)\
                                      - A0*n0*p0*(Omegan1-Omegap1)^2)\
              + 8*pi*(-Lambda2)\
              - 2/r^2*formal_diff(r/e^(lamb)*v0,r)\
              + r^2/(6*e^(lamb+nu))*(formal_diff(Ln,r))^2

for ii in range(len(sus_func)):
    ddEq_00 = ddEq_00.subs(sus_func[ii])
    ddEq_00_paper = ddEq_00_paper.subs(sus_func[ii])
    
for ii in range(0,len(sus_descomp),2):
    ddEq_00 = ddEq_00.substitute_function(sus_descomp[ii], sus_descomp[ii+1])
    ddEq_00_paper = ddEq_00_paper.substitute_function(sus_descomp[ii], sus_descomp[ii+1])

ddEq_00 = ddEq_00.simplify_full()
ddEq_00_paper = ddEq_00_paper.simplify_full()
for ii in range(len(sus_legendre)):
    ddEq_00 = ddEq_00.subs(sus_legendre[ii])
    ddEq_00_paper = ddEq_00_paper.subs(sus_legendre[ii])

ddEq_00 = ddEq_00.subs(LP2==0)
ddEq_00_paper = ddEq_00_paper.subs(LP2==0)
ddEq_00_check = ddEq_00 - ddEq_00_paper

show(LatexExpr("\\text{Ecuación (47) }\\rightarrow"), bool(ddEq_00_check==0))

show(EN(ddEq_00_check))

Ecuación (49)

In [ ]:
ddEq_11 = ddG[1,1] - kappa*ddT[1,1]

ddEq_11_paper = 2/(r*e^(lamb))*formal_diff(h0,r)\
              - 2/(r*e^(lamb))*(formal_diff(nu,r)+1/r)*v0\
              + r^2/(6*e^(nu+lamb))*formal_diff(Ln,r)^2\
              - 8*pi*(mu0*n0*gamman + chi0*p0*gammap\
                      - (Psi0 - Lambda0)*h0 + r^2/(3*e^(nu))*(mu0*n0*Ln^2 + chi0*p0*Lp^2)\
                      - r^2/(3*e^(nu))*n0*p0*A0*(Omegan1 - Omegap1)^2)

for ii in range(len(sus_func)):
    ddEq_11 = ddEq_11.subs(sus_func[ii])
    ddEq_11_paper = ddEq_11_paper.subs(sus_func[ii])
    
for ii in range(0,len(sus_descomp),2):
    ddEq_11 = ddEq_11.substitute_function(sus_descomp[ii], sus_descomp[ii+1])
    ddEq_11_paper = ddEq_11_paper.substitute_function(sus_descomp[ii], sus_descomp[ii+1])

ddEq_11 = ddEq_11.simplify_full()
ddEq_11_paper = ddEq_11_paper.simplify_full()
for ii in range(len(sus_legendre)):
    ddEq_11 = ddEq_11.subs(sus_legendre[ii])
    ddEq_11_paper = ddEq_11_paper.subs(sus_legendre[ii])

ddEq_11 = ddEq_11.subs(LP2==0)
ddEq_11_paper = ddEq_11_paper.subs(LP2==0)
ddEq_11_check = ddEq_11 - ddEq_11_paper

show(LatexExpr("\\text{Ecuación (49) }\\rightarrow"), bool(ddEq_11_check==0))

show(EN(ddEq_11_check))

### <center> $\boxed{l=2}$ </center>

Ecuación (39)

In [ ]:
Lambda2 = formal_diff(formal_diff(Lambdae, eps),eps).simplify_full()

for ii in range(len(sus_func)):
    Lambda2 = Lambda2.subs(sus_func[ii])
for ii in range(0, len(sus_descomp), 2):
    Lambda2 = Lambda2.substitute_function(sus_descomp[ii], sus_descomp[ii+1])
for ii in range(len(sus_legendre)):
    Lambda2 = Lambda2.subs(sus_legendre[ii])
Lambda22 = (Lambda2 - Lambda2.subs(LP2==0))/LP2

Lambda22_paper = mu0*n0*eta2 + chi0*p0*Phi2 - r^2/(3*e^(nu))*A0*n0*p0*(Omegan1-Omegap1)^2
Lambda22_paper = -Lambda22_paper
for ii in range(len(sus_func)):
    Lambda22_paper = Lambda22_paper.subs(sus_func[ii])

Lambda22_check = (Lambda22 - Lambda22_paper).simplify_full()

show(LatexExpr("\\Lambda^{(2)}_2 \\rightarrow"), bool(Lambda22_check==0))

show(EN(Lambda22_check))

Ecuación (43.1)

In [ ]:
Eq_43_1 = formal_diff(formal_diff(mue,eps),eps)/muinf
Eq_43_1_paper = + B000*n0*eta2/mu0\
                + A000*p0*Phi2/mu0\
                - r^2/(3*e^(nu))*p0/mu0*(A0+n0*formal_diff(A,n) + n0*p0*formal_diff(A,x2))*(Omegan1-Omegap1)^2\
                + r^2/(3*e^nu)*Ln^2 + h2
Eq_43_1_check = Eq_43_1 - Eq_43_1_paper
for ii in range(len(sus_func)):
    Eq_43_1 = Eq_43_1.subs(sus_func[ii])
    Eq_43_1_paper = Eq_43_1_paper.subs(sus_func[ii])
    Eq_43_1_check = Eq_43_1_check.subs(sus_func[ii])

Eq_43_1 = Eq_43_1.simplify_full()

for ii in range(len(sus_legendre)):
    Eq_43_1 = Eq_43_1.subs(sus_legendre[ii])
    Eq_43_1_paper = Eq_43_1_paper.subs(sus_legendre[ii])

Eq_43_1 = Eq_43_1.simplify_full()
Eq_43_1 = (Eq_43_1 - Eq_43_1.subs(LP2==0))/LP2
Eq_43_1_paper = (Eq_43_1_paper - Eq_43_1_paper.subs(LP2==0))/LP2
Eq_43_1 = Eq_43_1.simplify_full()

Eq_43_1_paper = Eq_43_1_paper.simplify_full()
Eq_43_1_check = Eq_43_1_check.simplify_full()

show(LatexExpr("\\text{Ecuación (43.1) }\\rightarrow"), bool(Eq_43_1_check==0))

show(EN(Eq_43_1_check))

Ecuación (43.2)

In [ ]:
var('gamman', latex_name = "\\gamma_n")

Eq_43_2 = formal_diff(formal_diff(mue,eps),eps)/muinf
Eq_43_2_paper = + C000*p0*Phi2/chi0\
                + A000*n0*eta2/chi0\
                - r^2/(3*e^(nu))*n0/chi0*(A0+p0*formal_diff(A,p) + n0*p0*formal_diff(A,x2))*(Omegan1-Omegap1)^2\
                + r^2/(3*e^nu)*Lp^2 + h2
Eq_43_2_check = Eq_43_2 - Eq_43_2_paper
for ii in range(len(sus_func)):
    Eq_43_2 = Eq_43_2.subs(sus_func[ii])
    Eq_43_2_paper = Eq_43_2_paper.subs(sus_func[ii])
    Eq_43_2_check = Eq_43_2_check.subs(sus_func[ii])

Eq_43_2 = Eq_43_2.simplify_full()

for ii in range(len(sus_legendre)):
    Eq_43_2 = Eq_43_2.subs(sus_legendre[ii])

Eq_43_2 = Eq_43_2.simplify_full()
Eq_43_2 = (Eq_43_2 - Eq_43_2.subs(LP2==0))/LP2
Eq_43_2 = Eq_43_2.simplify_full()

Eq_43_2_paper = Eq_43_2_paper.simplify_full()
Eq_43_2_paper = (Eq_43_2_paper - Eq_43_2_paper.subs(LP2==0))/LP2
Eq_43_2_paper = Eq_43_2_paper.simplify_full()

Eq_43_2_check = Eq_43_2_check.simplify_full()

show(LatexExpr("\\text{Ecuación (43.2) }\\rightarrow"), bool(Eq_43_2_check==0))

show(EN(Eq_43_2_check))

In [ ]:
sys.exit()

Ecuación (45)

In [ ]:
ddEq_2233 = ddG[2,2] - kappa*ddT[2,2] - ddG[3,3] + kappa*ddT[3,3] == 0

h2_value = solve(ddEq_2233, h2)[0].rhs()
h2_paper = r^4/(6*e^(nu+lamb))*formal_diff(Ln,r)^2 + 8*pi*r^4/(3*e^nu)*(Psi0-Lambda0)*Ln^2\
         + 8*pi*r^4/(3*e^(nu))*(chi0*p0*(Omegan1-Omegap1)*(Ln+Lp) - A0*n0*p0*(Omegan1-Omegap1)^2) - v2

for ii in range(len(sus_func)):
    h2_value = h2_value.subs(sus_func[ii])
    h2_paper = h2_paper.subs(sus_func[ii])

h2_value = h2_value.simplify_full()
h2_paper = h2_paper.simplify_full()
for ii in range(len(sus_legendre)):
    h2_value = h2_value.subs(sus_legendre[ii])
    h2_paper = h2_paper.subs(sus_legendre[ii]

h2_value = (h2_value - h2_value.subs(LP2==0))/LP2
h2_paper = (h2_paper - h2_paper.subs(LP2==0))/LP2
h2_check = (h2_value - h2_paper).simplify_full()

show(LatexExpr("h_2 \\rightarrow"), bool(h2_check==0))

show(EN(h2_check))

Ecuación (46)

In [ ]:
ddEq_12 = (ddG[1,2] - kappa*ddT[1,2])*(-r/(3*cos(th)*sin(th)))
ddEq_12_paper = (1/r*(v2+h2) - formal_diff(k2+h2,r) - formal_diff(nu,r)/2*(h2-v2))*2*r
for ii in range(len(sus_func)):
    ddEq_12 = ddEq_12.subs(sus_func[ii])
    ddEq_12_paper = ddEq_12_paper.subs(sus_func[ii])

ddEq_12_check = (ddEq_12 - ddEq_12_paper).simplify_full()

show(LatexExpr("\\text{Ecuación (46) }\\rightarrow"), bool(ddEq_12_check==0))

show(EN(ddEq_12_check))

Ecuación (48)

In [ ]:
ddEq_00 = ddG[0,0] - kappa*ddT[0,0]

ddEq_00_paper = - 16*pi*r^2/(3*e^(nu))*((Psi0 - Lambda0)*Ln^2\
                                      + chi0*p0*(Omegan1 - Omegap1)*(Ln + Lp)\
                                      - A0*n0*p0*(Omegan1 - Omegap1)^2)\
                + 8*pi*(- Lambda2)\
                - 2/r^2*formal_diff(r/e^(lamb)*v2,r)\
                - r^2/(6*e^(lamb + nu))*(formal_diff(Ln,r))^2\
                + 2/e^(lamb)*(formal_diff(formal_diff(k2,r),r)\
                              +(3/r - formal_diff(lamb,r)/2)*formal_diff(k2,r)\
                              - 2*e^(lamb)/r^2*k2) - 6/r^2*v2

for ii in range(len(sus_func)):
    ddEq_00 = ddEq_00.subs(sus_func[ii])
    ddEq_00_paper = ddEq_00_paper.subs(sus_func[ii])
    
for ii in range(0,len(sus_descomp),2):
    ddEq_00 = ddEq_00.substitute_function(sus_descomp[ii], sus_descomp[ii+1])
    ddEq_00_paper = ddEq_00_paper.substitute_function(sus_descomp[ii], sus_descomp[ii+1])

ddEq_00 = ddEq_00.simplify_full()
ddEq_00_paper = ddEq_00_paper.simplify_full()
for ii in range(len(sus_legendre)):
    ddEq_00 = ddEq_00.subs(sus_legendre[ii])
    ddEq_00_paper = ddEq_00_paper.subs(sus_legendre[ii])

ddEq_00 = (ddEq_00 - ddEq_00.subs(LP2==0))/LP2
ddEq_00 = ddEq_00.simplify_full()
ddEq_00_check = (ddEq_00 - ddEq_00_paper).simplify_full()


show(LatexExpr("\\text{Ecuación (48) }\\rightarrow"), bool(ddEq_00_check==0))

show(EN(ddEq_00_check))

Ecuación (50)

In [ ]:
ddEq_11 = ddG[1,1] - kappa*ddT[1,1]

ddEq_11_paper = 2/(r*e^(lamb))*formal_diff(h2,r)\
              - 6/r^2*h2 - 2/(r*e^(lamb))*(formal_diff(nu,r)+1/r)*v2\
              + 1/e^(lamb)*(formal_diff(nu,r)+2/r)*formal_diff(k2,r)\
              - 4/r^2*k2 - r^2/(6*e^(lamb + nu))*formal_diff(Ln,r)^2\
              + 8*pi*((Psi0 - Lambda0)*h2 + r^2/(3*e^(nu))*(mu0*n0*Ln^2 + chi0*p0*Lp^2)\
                      - r^2/(3*e^(nu))*n0*p0*A0*(Omegan1 - Omegap1)^2)

for ii in range(len(sus_func)):
    ddEq_11 = ddEq_11.subs(sus_func[ii])
    ddEq_11_paper = ddEq_11_paper.subs(sus_func[ii])

for ii in range(0,len(sus_descomp),2):
    ddEq_11 = ddEq_11.substitute_function(sus_descomp[ii], sus_descomp[ii+1])
    ddEq_11_paper = ddEq_11_paper.substitute_function(sus_descomp[ii], sus_descomp[ii+1])

ddEq_11 = ddEq_11.simplify_full()
ddEq_11_paper = ddEq_11_paper.simplify_full()
for ii in range(len(sus_legendre)):
    ddEq_11 = ddEq_11.subs(sus_legendre[ii])
    ddEq_11_paper = ddEq_11_paper.subs(sus_legendre[ii])

ddEq_11 = (ddEq_11 - ddEq_11.subs(LP2==0))/LP2
ddEq_11 = ddEq_11.simplify_full()
ddEq_11_paper = (ddEq_11_paper - ddEq_11_paper.subs(LP2==0))/LP2
ddEq_11_check = (ddEq_11 - ddEq_11_paper).simplify_full()

show(LatexExpr("\\text{Ecuación (50) }\\rightarrow"), bool(ddEq_11_check==0))

show(EN(ddEq_11_check))